In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('books.csv')
df

,isbn13,isbn10,title,subtitle,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count
0,9780002005883,0002005883,Gilead,NaN,Marilynne Robinson,Fiction,http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,2004.0,3.85,247.0,361.0
1,9780002261982,0002261987,Spider's Web,A Novel,Charles Osborne;Agatha Christie,Detective and mystery stories,http://books.google.com/books/content?id=gA5GP...,A new 'Christie for Christmas' -- a full-lengt...,2000.0,3.83,241.0,5164.0
2,9780006163831,0006163831,The One Tree,NaN,Stephen R. Donaldson,American fiction,http://books.google.com/books/content?id=OmQaw...,Volume Two of Stephen Donaldson's acclaimed se...,1982.0,3.97,479.0,172.0
3,9780006178736,0006178731,Rage of angels,NaN,Sidney Sheldon,Fiction,http://books.google.com/books/content?id=FKo2T...,"A memorable, mesmerizing heroine Jennifer -- b...",1993.0,3.93,512.0,29532.0
4,9780006280897,0006280897,The Four Loves,NaN,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=XhQ5X...,Lewis' work on the nature of love divides love...,2002.0,4.15,170.0,33684.0
...,...,...,...,...,...,...,...,...,...,...,...,...
6805,9788185300535,8185300534,I Am that,Talks with Sri Nisargadatta Maharaj,Sri Nisargadatta Maharaj;Sudhakar S. Dikshit,Philosophy,http://books.google.com/books/content?id=Fv_JP...,This collection of the timeless teachings of o...,1999.0,4.51,531.0,104.0
6806,9788185944609,8185944601,Secrets Of The Heart,NaN,Khalil Gibran,Mysticism,http://books.google.com/books/content?id=XcrVp...,NaN,1993.0,4.08,74.0,324.0
6807,9788445074879,8445074873,Fahrenheit 451,NaN,Ray Bradbury,Book burning,NaN,NaN,2004.0,3.98,186.0,5733.0
6808,9789027712059,9027712050,The Berlin Phenomenology,NaN,Georg Wilhelm Friedrich Hegel,History,http://books.google.com/books/content?id=Vy7Sk...,Since the three volume edition ofHegel's Philo...,1981.0,0.00,210.0,0.0


In [3]:
def textual_representation(row):
    textual_representation = f"""Title: {row['title']}
Authors: {row['authors']} 
Description: {row['description']}
Categories: {row['categories']}
Publishing Year: {row['published_year']}
Average Rating: {row['average_rating']}
Number of Pages: {row['num_pages']}
"""
    
# Alternative structure   
#        textual_representation = f"""Categories: {row['categories']}
#Title: {row['title']}
#Authors: {row['authors']} 
#Average Rating: {row['average_rating']}
#Number of Pages: {row['num_pages']}
#Publishing Year: {row['published_year']}

#Description: {row['description']}
#"""
    
    return textual_representation

In [4]:
print(df.iloc[:5].apply(textual_representation, axis=1).values[0])

Title: Gilead
Authors: Marilynne Robinson 
Description: A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives. John Ames is a preacher, the son of a preacher and the grandson (both maternal and paternal) of preachers. It’s 1956 in Gilead, Iowa, towards the end of the Reverend Ames’s life, and he is absorbed in recording his family’s story, a legacy for the young son he will never see grow up. Haunted by his grandfather’s presence, John tells of the rift between his grandfather and his father: the elder, an angry visionary who fought for the abolitionist cause, and his son, an ardent pacifist. He is troubled, too, by his prodigal namesake, Jack (John Ames) Boughton, his best friend’s lost son who returns to Gilead searching for forgiveness and redemption. Told in John Ames’s joyous, rambling voice that finds beauty, humour and truth in the smallest of life’s details, Gilead is a song of celebratio

In [5]:
df['textual_representation'] = df.apply(textual_representation, axis=1)

In [7]:
import faiss
import requests
import numpy as np

print(faiss.__version__)


# Also ollama run and ollama pull llama2 commands are required to run ollama model
 
dim = 4096 

index = faiss.IndexFlatL2(dim) # VectorStore with dimensanality. We choose this dimensionality becaues it is in response we get from Llama 2

X = np.zeros((len(df['textual_representation']), dim), dtype='float32')


1.8.0


In [11]:
for i, representation in enumerate(df['textual_representation']):
    res = requests.post('http://localhost:11434/api/embeddings', # Default port of Ollama. Also we can use OpenAI API embeddings, but it's not for free
                        json = {
                            'model': 'llama3.1',
                            'prompt': representation
                        }) 
    
    embedding = res.json()['embedding']
    #embedding = res.json()

    X[i] = np.array(embedding)

index.add(X)

# It takes some time

KeyboardInterrupt: 

In [ ]:
faiss.write_index(index, 'index')

In [ ]:
index = faiss.read_index('index') # The result of embedding

In [ ]:
df[df.title.str.contains('friends')]

In [ ]:
favorite_book = df.iloc[4533]
print(favorite_book['textual_representation'])

In [ ]:
# Now we can use vector store to find similar books
res = requests.post('http://localhoste:11434/api/embeddings', # Default port of Ollama. Also we can use OpenAI API embeddings, but it's not for free
                        json = {
                            'model': 'llama2',
                            'prompt': favorite_book['textual_representation']
                        }) 

embedding = np.array([res.json()['embedding']], dtype='float32')

In [ ]:
D, I = index.search(embedding, 5)

In [ ]:
best_matches = np.array(df['textual_representation'])[I.flatten()]

In [ ]:
for match in best_matches:
    print(match)
    print()